Exercise 1
Create table orders_part with the same columns as orders.

Partition the table by month using range partitioning on order_date.

Add 14 partitions - 13 based up on the data and 1 default. Here is the naming convention.

Default - orders_part_default

Partition for 2014 January - orders_part_201401

```sql
CREATE TABLE orders_part (
  order_id INT NOT NULL,
  order_date TIMESTAMP NOT NULL,
  order_customer_id INT NOT NULL,
  order_status VARCHAR(45) NOT NULL,
  PRIMARY KEY (order_date,order_id)
) PARTITION BY RANGE(order_date);

CREATE TABLE orders_part_default
PARTITION OF orders_part DEFAULT;
```

In [40]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas

connection = psycopg2.connect(
    host="localhost",
    database="itversity_retail_db",
    user="itversity_retail_user",
    password="retail_password"
)

connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

cursor = connection.cursor()

dates_start = pandas.date_range('2013-07', periods=13, freq='MS')
dates_end = pandas.date_range('2013-08', periods=13, freq='MS')

zip_dates = zip(dates_start, dates_end)

for date_start,date_end in zip_dates:
    cursor.execute(f'''
        CREATE TABLE orders_part_{date_start.strftime("%Y%m")}
        PARTITION OF orders_part
        FOR VALUES FROM ('{date_start.strftime("%Y-%m-%d")}') TO ('{date_end.strftime("%Y-%m-%d")}')
    ''')

cursor.close()
connection.close()

Exercise 2¶
Let us load and validate data in the partitioned table.

Load the data from orders into orders_part.

Get count on orders_part as well as all the 14 partitions. You should get 0 for default partition and all the records should be distributed using the other 13 partitions.

```sql
INSERT INTO orders_part
SELECT * FROM orders;

SELECT count(*) FROM orders_part_default;

SELECT count(*) FROM orders_part_201307;
```